In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.6.0
sys.version_info(major=3, minor=8, micro=3, releaselevel='final', serial=0)
matplotlib 3.3.4
numpy 1.19.5
pandas 1.3.4
sklearn 0.22.1
tensorflow 2.6.0
keras.api._v2.keras 2.6.0


In [4]:
def f(x):
    return 3. * x ** 2 + 2. * x - 1
#近视求导，x移动eps单位，也就是离自己很近的一个点的切线
def approximate_derivative(f, x, eps=1e-4):
    return (f(x + eps) - f(x - eps)) / (2. * eps)

print(approximate_derivative(f, 1.))

7.999999999994678


In [5]:
#求偏导数
def g(x1, x2):
    return (x1 + 5) * (x2 ** 2)

def approximate_gradient(g, x1, x2, eps=1e-3):
    dg_x1 = approximate_derivative(lambda x: g(x, x2), x1, eps)
    dg_x2 = approximate_derivative(lambda x: g(x1, x), x2, eps)
    return dg_x1, dg_x2

print(approximate_gradient(g, 2., 3.))
    

(8.999999999993236, 41.999999999994486)


In [8]:
#在tf中tape只能用一次
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)
with tf.GradientTape() as tape:
    z = g(x1, x2)

#求偏导
dz_x1 = tape.gradient(z, x1)
print(dz_x1)
print(x1)
try:
    dz_x2 = tape.gradient(z, x2)
except RuntimeError as ex:
    print(ex)

tf.Tensor(9.0, shape=(), dtype=float32)
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>
A non-persistent GradientTape can only be used to compute one set of gradients (or jacobians)


In [6]:
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)
#加入persistent后，可以让tape使用一次后不释放
with tf.GradientTape(persistent = True) as tape:
    z = g(x1, x2)

dz_x1 = tape.gradient(z, x1)
dz_x2 = tape.gradient(z, x2)
print(dz_x1, dz_x2)
#要手动释放
del tape

tf.Tensor(9.0, shape=(), dtype=float32) tf.Tensor(42.0, shape=(), dtype=float32)


In [7]:
#同时求偏导，传入列表
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)
with tf.GradientTape() as tape:
    z = g(x1, x2)

dz_x1x2 = tape.gradient(z, [x1, x2])

print(dz_x1x2)
print(type(dz_x1x2))

[<tf.Tensor: shape=(), dtype=float32, numpy=9.0>, <tf.Tensor: shape=(), dtype=float32, numpy=42.0>]
<class 'list'>


In [17]:
#同时求偏导，传入列表，可以在with中编写多个公式
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)
with tf.GradientTape() as tape:
    z = g(x1, x2)
    z1=z+3

dz1_x1= tape.gradient(z1, x1)

print(dz1_x1)

tf.Tensor(9.0, shape=(), dtype=float32)


In [8]:
#constant是常量，我们可以对常量求偏导么？
x1 = tf.constant(2.0)
x2 = tf.constant(3.0)
with tf.GradientTape() as tape:
    z = g(x1, x2)

dz_x1x2 = tape.gradient(z, [x1, x2])

print(dz_x1x2)

[None, None]


In [9]:
#如何对常量求偏导，我们关注常量的导数
x1 = tf.constant(2.0)
x2 = tf.constant(3.0)
with tf.GradientTape() as tape:
    tape.watch(x1)#在这里增加watch
    tape.watch(x2)
    z = g(x1, x2)

dz_x1x2 = tape.gradient(z, [x1, x2])

print(dz_x1x2)

[<tf.Tensor: shape=(), dtype=float32, numpy=9.0>, <tf.Tensor: shape=(), dtype=float32, numpy=42.0>]


In [10]:
#两个目标函数对一个变量求导数
x = tf.Variable(5.0)
with tf.GradientTape() as tape:
    z1 = 3 * x
    z2 = x ** 2
    z3 = x ** 3
tape.gradient([z1, z2,z3], x)   #3+10+75，三个结果相加，而不是分开

<tf.Tensor: shape=(), dtype=float32, numpy=88.0>

In [11]:
#如何求二阶导数呢？
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)

#return (x1 + 5) * (x2 ** 2)
#因为会执行多次，所以加入persistent
with tf.GradientTape(persistent=True) as outer_tape:
    with tf.GradientTape(persistent=True) as inner_tape:
        z = g(x1, x2)
    inner_grads = inner_tape.gradient(z, [x1, x2]) #inner_grads是一个列表，包含x1和x2的导数。。。
outer_grads = [outer_tape.gradient(inner_grad, [x1, x2])  # 对这些导数分别再对x1和x2求导
               for inner_grad in inner_grads]
print(outer_grads)
del inner_tape
del outer_tape
#结果依次是None，先对x2求导，在对x1求导，接着是反过来，接着是对x2求2阶导数

[[None, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>], [<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, <tf.Tensor: shape=(), dtype=float32, numpy=14.0>]]


In [13]:
#模拟梯度下降算法 SGD
learning_rate = 0.1
#随意给一个x,就像我们的w一开始给的是格罗特均匀分布
x = tf.Variable(-5.0)

for _ in range(100):
    with tf.GradientTape() as tape:
        z = f(x)
    dz_dx = tape.gradient(z, x)
    #求得导数后对x建更新
    x.assign_sub(learning_rate * dz_dx)
print(x)  #就是y最小的时候，-b/2*a

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-0.33333334>


In [14]:
#GradientTape与optimizer（优化器）结合使用
learning_rate = 0.1
x = tf.Variable(2.0)

optimizer = keras.optimizers.SGD(lr = learning_rate)

for _ in range(100):
    with tf.GradientTape() as tape:
        z = f(x)
    dz_dx = tape.gradient(z, x)
    optimizer.apply_gradients([(dz_dx, x)])
print(x)
#不同优化器的区别就在于learning_rate的变化规律是不一样的

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-0.3333333>


In [9]:
list(zip((1,2,3),(4,5,6))) # zip将两个列表对应起来

[(1, 4), (2, 5), (3, 6)]